In [1]:
!pip install python-dotenv
!pip install openai

# Getting data from Mergeflow's API

In [20]:
import requests
import json
import os
from dotenv import load_dotenv

load_dotenv()  # Load variables from .env file
mergeflow_api_key = os.getenv('MERGEFLOW_API_KEY')

# the API call
query = 'Factory+Automation' # topic is "large language model"
dataset = '&q=*[MarketSize]' # dataset: Tech blogs
rows = str(3) # first 3 docs only

# in the url below, replace "fw" by your user name. You can see your user name at mergeflow.net
# -> Account -> the string in angle brackets
url = 'https://mergeflow.net/api/v1/lutzenrath/search/getContent?q=' + query + dataset + '&rows=' + rows

# Define the headers to be sent with the request
headers = {
    'MergeflowNet-Api-Auth-Key': mergeflow_api_key
}

# Get and print the response
response = requests.post(url, headers=headers)

mergeflow_docs = []

# Check the response status code and content
if response.status_code == 200:
    mergeflow_response_json = response.json()
    
    # Pretty-print the JSON response -- useful to understand what the JSON looks like
    #pretty_json = json.dumps(mergeflow_response_json, indent=4)
    #print(pretty_json)
    
    # get title, URL, date, and content
    for document in mergeflow_response_json['Documents']:
        current_doc = {
            'title': document['Title'],
            'date': document['Date'],
            'url': document['Url'],
            'content': document['Content'][:5000]
        }
        
        mergeflow_docs.append(current_doc)
        #print(current_doc['content'])        

    print('Found ' + str(len(mergeflow_docs)) + ' documents.')

else:
    print('Request failed with status code:', response.status_code)
    print(response.text)  # Print the response text


Found 3 documents.


# Answering questions on the results with OpenAI GPT

In [21]:
import openai

openai.api_key = os.getenv('OPENAI_API_KEY')

system_prompt = f"""You are Senior Vice President Technology and Innovation & CTO of Factory Automation. When you evaluate a new business idea, you strictly adhere to the following guiding principles:
        - Prioritize quick, low-investment idea testing.
        - Aim for large profits with smaller budgets, avoiding investments over 10-15 million per portfolio element.
        - Target a minimum 10 million EUR annual revenue.
        - Avoid new ecosystem creation.
        - Prioritize product over solution business for scalability.
        - Consider AI as a potential product.
        - Ensure all initiatives align with business profitability.
        - Make investment decisions as if using personal funds.
        - Aim for non-cannibalizing, fringe markets.
        """

for doc in mergeflow_docs:
    user_prompt = f"""
    Consider the text below, delimited by ```, in the area of {query}:
    ```
    {doc['content']}
    ```
    Focus on the three markets with the largest CAGR. Come up with business opportunities in those markets and rank them on a scale from 1 (bad) to 3(good). Explain your reasoning in 30 words or less.
    
    """
    
    completion = openai.chat.completions.create(
        model="gpt-3.5-turbo-0613",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.7,
        max_tokens=2000,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )

    print(completion.choices[0].message)

ChatCompletionMessage(content='Based on the information provided, the three markets with the largest CAGR in the Smart Factory industry are:\n1. Industrial IoT (IIoT) - Opportunities: AI-based predictive maintenance solutions, IIoT platforms for data analytics, and AI-powered quality control systems. Rank: 3 (good) - high growth potential and aligns with AI as a potential product.\n2. Machine Learning (ML) - Opportunities: ML algorithms for demand forecasting, ML-based optimization of production processes, and ML-powered defect detection systems. Rank: 3 (good) - high growth potential and aligns with AI as a potential product.\n3. Industry 4.0 - Opportunities: Cloud-based automation systems, robotic process automation (RPA) solutions, and digital twin technology for virtual simulation. Rank: 2 (average) - moderate growth potential and may require more investment in ecosystem creation.', role='assistant', function_call=None, tool_calls=None)
ChatCompletionMessage(content='Based on the p